## 파일 다운로드

In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import plot_importance
import os

# CSV 파일 경로
base_path = r"C:\Users\bok7z\DACSV"

# CSV 파일 읽기
patients = pd.read_csv("./data/patients.csv")
admissions = pd.read_csv("./data/admissions.csv")
procedures = pd.read_csv("./data/procedures_icd.csv")
d_icd_procedures = pd.read_csv("./data/d_icd_procedures.csv")


## 데이터 전처리: 정맥 카테터 삽입 수술 건 필터링

In [2]:
# 1. 환자와 입원 데이터 조인
patients_subset = patients[["subject_id", "gender", "anchor_age"]]
admissions_subset = admissions[[
    "subject_id", "hadm_id", "admission_type", "insurance", "admittime", "dischtime"
]]
patient_admission = pd.merge(admissions_subset, patients_subset, on="subject_id", how="inner")

# 2. 수술 데이터 조인
procedures_subset = procedures[["hadm_id", "icd_code", "seq_num"]]
surgery_data = pd.merge(patient_admission, procedures_subset, on="hadm_id", how="left")

# 3. 수술 코드 설명 매핑
icd_mapping = d_icd_procedures[["icd_code", "long_title"]]
surgery_with_description = pd.merge(surgery_data, icd_mapping, on="icd_code", how="left")

# 4. 특정 수술 필터링
enteral_infusion_data = surgery_with_description[
    surgery_with_description["long_title"] == "Venous catheterization, not elsewhere classified"
]

# 날짜 데이터 처리
enteral_infusion_data['admittime'] = pd.to_datetime(enteral_infusion_data['admittime'])
enteral_infusion_data['dischtime'] = pd.to_datetime(enteral_infusion_data['dischtime'])

# 재입원 여부 계산
enteral_infusion_data['time_to_next_admission'] = enteral_infusion_data.groupby('subject_id')['admittime'].shift(-1) - enteral_infusion_data['dischtime']
enteral_infusion_data['time_to_next_admission'] = enteral_infusion_data['time_to_next_admission'].fillna(pd.Timedelta(days=0))
enteral_infusion_data['readmission_within_30_days'] = enteral_infusion_data['time_to_next_admission'].apply(
    lambda x: 1 if pd.notnull(x) and x <= pd.Timedelta(days=30) else 0
)

# 범주형 데이터 변환
enteral_infusion_data['long_title'] = enteral_infusion_data['long_title'].astype('category').cat.codes
enteral_infusion_data['admission_type'] = enteral_infusion_data['admission_type'].astype('category').cat.codes
enteral_infusion_data['insurance'] = enteral_infusion_data['insurance'].astype('category').cat.codes
enteral_infusion_data['gender'] = enteral_infusion_data['gender'].astype('category').cat.codes
enteral_infusion_data['icd_code'] = enteral_infusion_data['icd_code'].astype('category').cat.codes

/var/folders/db/vpd2vbln62d_v_cvwyfcw3z00000gn/T/ipykernel_15300/505399355.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enteral_infusion_data['admittime'] = pd.to_datetime(enteral_infusion_data['admittime'])
/var/folders/db/vpd2vbln62d_v_cvwyfcw3z00000gn/T/ipykernel_15300/505399355.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enteral_infusion_data['dischtime'] = pd.to_datetime(enteral_infusion_data['dischtime'])
/var/folders/db/vpd2vbln62d_v_cvwyfcw3z00000gn/T/ipykernel_15300/505399355.py:26

In [10]:
enteral_infusion_data.to_csv("./data/enteral_infusion_data.csv", index=False)

## NaN 처리
enteral_infusion_data = enteral_infusion_data.fillna(0)

## 데이터 전처리
1. icd_version이 9와 10인 icd_code 병합
2. icd_code_9와 icd_code_10을 감염성 질환, 신장계, 순환계, 혈액암/림프종, 호흡기, 혈액응고, 자가면역으로 세분화하여 변수 생성
3. icd_code_9와 icd_code_10은 삭제
4. 감염성 질환, 신장계, 순환계, 혈액암/림프종, 호흡기, 혈액응고, 자가면역에 해당하는 사람은 해당 변수 값을 1, 아닌 사람은 0으로 초기화

In [12]:
# diagnoses_icd.csv의 icd_version이 9, 10인 icd_code들을 enteral_infusion_data.csv에 join

# 데이터 불러오기
enteral_infusion_data = pd.read_csv('./data/enteral_infusion_data')
diagnoses_icd = pd.read_csv('./data/diagnoses_icd.csv')


## 질병 데이터 추출 전처리

In [14]:
# icd_version이 9인 데이터만 추출
diagnoses_icd_9 = diagnoses_icd[diagnoses_icd['icd_version'] == 9][['subject_id', 'icd_code']]
diagnoses_icd_9 = diagnoses_icd_9.rename(columns={'icd_code': 'icd_code_9'})

# icd_version이 10인 데이터만 추출
diagnoses_icd_10 = diagnoses_icd[diagnoses_icd['icd_version'] == 10][['subject_id', 'icd_code']]
diagnoses_icd_10 = diagnoses_icd_10.rename(columns={'icd_code': 'icd_code_10'})

# enteral_infusion_data와 icd_version 9 데이터를 subject_id 기준으로 조인
enteral_infusion_data_9 = pd.merge(enteral_infusion_data, diagnoses_icd_9, on='subject_id', how='left')

# enteral_infusion_data와 icd_version 10 데이터를 subject_id 기준으로 조인
enteral_infusion_data_10 = pd.merge(enteral_infusion_data_9, diagnoses_icd_10, on='subject_id', how='left')

# 결과 확인
print(enteral_infusion_data_10.head())

   subject_id   hadm_id  admission_type  insurance            admittime  \
0    11785054  26269995               3          2  2115-02-08 17:52:00   
1    11785054  26269995               3          2  2115-02-08 17:52:00   
2    11785054  26269995               3          2  2115-02-08 17:52:00   
3    11785054  26269995               3          2  2115-02-08 17:52:00   
4    11785054  26269995               3          2  2115-02-08 17:52:00   

             dischtime  gender  anchor_age  icd_code  seq_num  long_title  \
0  2115-03-24 11:30:00       1           0         0      2.0           0   
1  2115-03-24 11:30:00       1           0         0      2.0           0   
2  2115-03-24 11:30:00       1           0         0      2.0           0   
3  2115-03-24 11:30:00       1           0         0      2.0           0   
4  2115-03-24 11:30:00       1           0         0      2.0           0   

  time_to_next_admission  readmission_within_30_days icd_code_9 icd_code_10  
0       

In [27]:
# subject_id의 누락 없이 성공적으로 잘 join이 된 것을 확인

print("병합 전 enteral_infusion_data의 subject_id 개수:", enteral_infusion_data['subject_id'].nunique())
print("병합 후 enteral_infusion_data_10의 subject_id 개수:", enteral_infusion_data_10['subject_id'].nunique())


병합 전 enteral_infusion_data의 subject_id 개수: 10840
병합 후 enteral_infusion_data_10의 subject_id 개수: 10840


In [28]:
# 질병 카테고리와 ICD 코드 범위 정의
disease_categories = {
    "infectious_disease": {  # 감염성 질환
        "icd_9": range(1, 40),  # 001~039
        "icd_10": ["A" + str(i).zfill(2) for i in range(0, 100)],  # A00~B99
    },
    "renal_system": {  # 신장계
        "icd_9": range(590, 594),  # 590~593
        "icd_10": ["N" + str(i).zfill(2) for i in range(25, 30)],  # N25~N29
    },
    "circulatory_system": {  # 순환계
        "icd_9": range(390, 460),  # 390~459
        "icd_10": ["I" + str(i).zfill(2) for i in range(0, 100)],  # I00~I99
    },
    "hematologic_lymphatic": {  # 혈액암/림프종
        "icd_9": range(200, 209),  # 200~208
        "icd_10": ["C" + str(i).zfill(2) for i in range(81, 96)],  # C81~C95
    },
    "respiratory_system": {  # 호흡기
        "icd_9": range(460, 520),  # 460~519
        "icd_10": ["J" + str(i).zfill(2) for i in range(0, 100)],  # J00~J99
    },
    "coagulation": {  # 혈액응고
        "icd_9": [286],  # 286
        "icd_10": ["D" + str(i).zfill(2) for i in range(65, 70)],  # D65~D69
    },
    "immune_system": {  # 자가면역
        "icd_9": [279],  # 279
        "icd_10": ["D" + str(i).zfill(2) for i in range(80, 90)],  # D80~D89
    }
}

In [29]:

# 질병 카테고리에 대한 새로운 변수 생성
for category, codes in disease_categories.items():
    # ICD-9 처리
    condition_9 = enteral_infusion_data_10['icd_code_9'].apply(
        lambda x: 1 if pd.notnull(x) and x.isdigit() and int(x) in codes["icd_9"] else 0
    )
    # ICD-10 처리
    condition_10 = enteral_infusion_data_10['icd_code_10'].apply(
        lambda x: 1 if pd.notnull(x) and any(x.startswith(code[:1]) and code[1:] in x for code in codes["icd_10"]) else 0
    )
    # 두 조건 합쳐서 최종 변수 생성
    enteral_infusion_data_10[category] = condition_9 | condition_10

# 결과 확인
print(enteral_infusion_data_10.head())

   subject_id   hadm_id  admission_type  insurance            admittime  \
0    11785054  26269995               3          2  2115-02-08 17:52:00   
1    11785054  26269995               3          2  2115-02-08 17:52:00   
2    11785054  26269995               3          2  2115-02-08 17:52:00   
3    11785054  26269995               3          2  2115-02-08 17:52:00   
4    11785054  26269995               3          2  2115-02-08 17:52:00   

             dischtime  gender  anchor_age  icd_code  seq_num  ...  \
0  2115-03-24 11:30:00       1           0         0      2.0  ...   
1  2115-03-24 11:30:00       1           0         0      2.0  ...   
2  2115-03-24 11:30:00       1           0         0      2.0  ...   
3  2115-03-24 11:30:00       1           0         0      2.0  ...   
4  2115-03-24 11:30:00       1           0         0      2.0  ...   

   readmission_within_30_days icd_code_9  icd_code_10 infectious_disease  \
0                           1       V290          Na

In [30]:

# 결과를 CSV 파일로 저장
enteral_infusion_data_10.to_csv('enteral_infusion_data_with_disease_flags_final.csv', index=False)

In [31]:
enteral_infusion_data_10

,subject_id,hadm_id,admission_type,insurance,admittime,dischtime,gender,anchor_age,icd_code,seq_num,...,readmission_within_30_days,icd_code_9,icd_code_10,infectious_disease,renal_system,circulatory_system,hematologic_lymphatic,respiratory_system,coagulation,immune_system
0,11785054,26269995,3,2,2115-02-08 17:52:00,2115-03-24 11:30:00,1,0,0,2.0,...,1,V290,NaN,0,0,0,0,0,0,0
1,11785054,26269995,3,2,2115-02-08 17:52:00,2115-03-24 11:30:00,1,0,0,2.0,...,1,77989,NaN,0,0,0,0,0,0,0
2,11785054,26269995,3,2,2115-02-08 17:52:00,2115-03-24 11:30:00,1,0,0,2.0,...,1,V053,NaN,0,0,0,0,0,0,0
3,11785054,26269995,3,2,2115-02-08 17:52:00,2115-03-24 11:30:00,1,0,0,2.0,...,1,36224,NaN,0,0,0,0,0,0,0
4,11785054,26269995,3,2,2115-02-08 17:52:00,2115-03-24 11:30:00,1,0,0,2.0,...,1,28749,NaN,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70634347,16624205,23962991,5,2,2126-02-03 15:27:00,2126-03-18 16:15:00,0,44,0,7.0,...,1,4532,NaN,0,0,0,0,0,0,0
70634348,16624205,23962991,5,2,2126-02-03 15:27:00,2126-03-18 16:15:00,0,44,0,7.0,...,1,43820,NaN,0,0,0,0,0,0,0
70634349,16624205,23962991,5,2,2126-02-03 15:27:00,2126-03-18 16:15:00,0,44,0,7.0,...,1,27542,NaN,0,0,0,0,0,0,0
70634350,16624205,23962991,5,2,2126-02-03 15:27:00,2126-03-18 16:15:00,0,44,0,7.0,...,1,24900,NaN,0,0,0,0,0,0,0


In [32]:
# icd_code_9와 icd_code_10 변수 제거
enteral_infusion_data_10 = enteral_infusion_data_10.drop(columns=['icd_code_9', 'icd_code_10'])
# 결과 확인
print(enteral_infusion_data_10.head())


   subject_id   hadm_id  admission_type  insurance            admittime  \
0    11785054  26269995               3          2  2115-02-08 17:52:00   
1    11785054  26269995               3          2  2115-02-08 17:52:00   
2    11785054  26269995               3          2  2115-02-08 17:52:00   
3    11785054  26269995               3          2  2115-02-08 17:52:00   
4    11785054  26269995               3          2  2115-02-08 17:52:00   

             dischtime  gender  anchor_age  icd_code  seq_num  long_title  \
0  2115-03-24 11:30:00       1           0         0      2.0           0   
1  2115-03-24 11:30:00       1           0         0      2.0           0   
2  2115-03-24 11:30:00       1           0         0      2.0           0   
3  2115-03-24 11:30:00       1           0         0      2.0           0   
4  2115-03-24 11:30:00       1           0         0      2.0           0   

  time_to_next_admission  readmission_within_30_days  infectious_disease  \
0        0

In [33]:
enteral_infusion_data_10.to_csv('enteral_infusion_data_without_icd_codes.csv', index=False)

## hadm_id 기준으로 중복 샘플을 집계(aggregate)
hadm_id가 동일한 여러 샘플이 있으면, 질병 정보(원핫 인코딩)를 합쳐서 하나의 샘플로 만든다.

In [ ]:
b = enteral_infusion_data_10.drop_duplicates(keep='first')
b_grouped = b.groupby("hadm_id").max().reset_index()

## 최종 전처리된 데이터 저장
파일명: enteral_data_final.csv

In [ ]:
b_grouped.to_csv("./data/enteral_data_final.csv", index = False)

### 